In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{game}_upcache', connect_args={'connect_timeout': 10})

def get_recent_purchases_sql():
    return """
            select time, game, CONCAT(tax_country,'(',country_tier,')') AS `country`, description,
            currency, amt AS `price`, user_id, ROUND(prev_receipts,2) AS `prev_rcpt` from
            (select time, 'MF' as game, country_tier, tax_country, description, currency, round(currency_amount,2) AS amt, user_id, prev_receipts, IF(prev_receipts<0.01,1,0) AS first_purchase from mf_upcache.mf_credits where time >= UNIX_TIMESTAMP()-86400 and usd_receipts_cents > 0
            UNION ALL
            select time, 'TR' as game, country_tier, tax_country, description, currency, round(currency_amount,2) AS amt, user_id, prev_receipts, IF(prev_receipts<0.01,1,0) AS first_purchase from tr_upcache.tr_credits where time >= UNIX_TIMESTAMP()-86400 and usd_receipts_cents > 0
            UNION ALL
            select time, 'WSE' as game, country_tier, tax_country, description, currency, round(currency_amount,2) AS amt, user_id, prev_receipts, IF(prev_receipts<0.01,1,0) AS first_purchase from mf2_upcache.mf2_credits where time >= UNIX_TIMESTAMP()-86400 and usd_receipts_cents > 0
            UNION ALL
            select time, 'BFM' as game, country_tier, tax_country, description, currency, round(currency_amount,2) AS amt, user_id, prev_receipts, IF(prev_receipts<0.01,1,0) AS first_purchase from bfm_upcache.bfm_credits where time >= UNIX_TIMESTAMP()-86400 and usd_receipts_cents > 0
            UNION ALL
            select time, 'DV' as game, country_tier, tax_country, description, currency, round(currency_amount,2) AS amt, user_id, prev_receipts, IF(prev_receipts<0.01,1,0) AS first_purchase from dv_upcache.dv_credits where time >= UNIX_TIMESTAMP()-86400 and usd_receipts_cents > 0
            UNION ALL
            select time, 'FS' as game, country_tier, tax_country, description, currency, round(currency_amount,2) AS amt, user_id, prev_receipts, IF(prev_receipts<0.01,1,0) AS first_purchase from fs_upcache.fs_credits where time >= UNIX_TIMESTAMP()-86400 and usd_receipts_cents > 0
            ) temp order by time desc;
           """
print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

Dashboard updated Sun, 23 Jan 2022 at 20:35:39 UTC


In [14]:
# Incidents vs. Tier 1/2 and superfan defenders

engine = get_engine('tr')
recent_purchases_sql = get_recent_purchases_sql()
recent_purchases_data = pd.read_sql(recent_purchases_sql, engine)
if len(recent_purchases_data) > 0:
    print(f'\nRecent Purchases\n')
    header_line = "{0:20} {1:5} {2:8} {3:14} {4:12} {5:8} {6:8} {7:8}".format('Time', 'Game', 'Country', 'Description', 'Currency', 'Price', 'User ID', 'Previous Receipts')
    print(header_line)
    for i, timestamp in enumerate(recent_purchases_data['time']):
        formatted_date = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        game = recent_purchases_data['game'][i].upper()
        country = recent_purchases_data['country'][i]
        description = recent_purchases_data['description'][i]
        currency = recent_purchases_data['currency'][i]
        price = '$' + str(recent_purchases_data['price'][i])
        user_id = recent_purchases_data['user_id'][i]
        previous_receipts = '$' + str(recent_purchases_data['prev_rcpt'][i])
        if description.endswith(',firstbuy'):
            description = 'Item Bundle (firstbuy)'
        elif description.endswith(',rebuy'):
            description = 'Item Bundle (rebuy)'
        elif 'FLASH_25' in description or 'FLASH25' in description:
            description = '75% Off'
        elif 'FLASH_50' in description or 'FLASH50' in description:
            description = '50% off'
        elif description.endswith(',tokens'):
            description = 'ONP Bundle',
        elif description.endswith(',lottery'):
            description ='War Chest Bundle',
        elif 'D2' in description or description.startswith('FB_GAMEBUCKS_PAYMENT'):
            description = '-'
        output_line = "{0:20} {1:5} {2:8} {3:14} {4:12} {5:8} {6:8} {7:8}".format(str(formatted_date), str(game), str(country), str(description), str(currency), str(price), str(user_id), str(previous_receipts)
        print(output_line)
else:
    print(f'\nChecked recent purchases since, but none found.')


Recent Purchases

Time                 Game  Country  Description    Currency     Price    User ID  Previous Receipts
2022-01-23 19:37:42  TR    US(2)    -              USD          $9.99    668920   $12509.49
2022-01-23 19:24:05  TR    AU(1)    -              AUD          $99.99   4084562  $3482.4 
2022-01-23 19:04:26  FS    None     -              xsolla:GBP   $24.99   1194022  $86.44  
2022-01-23 18:30:09  TR    US(2)    -              USD          $4.99    3621840  $3823.49
2022-01-23 18:04:37  TR    US(2)    -              USD          $4.99    4183767  $2487.86
2022-01-23 17:54:26  TR    US(2)    -              USD          $9.99    668920   $12502.5
2022-01-23 17:31:29  TR    US(2)    -              USD          $9.99    3966534  $10263.1
2022-01-23 17:31:19  TR    None     -              kgcredits    $49.0    4227010  $467.46 
2022-01-23 16:45:43  DV    IT(3)    -              EUR          $9.99    376118   $853.17 
2022-01-23 16:17:59  DV    GB(1)    75% Off        GBP       